In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import mnist
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split

device = "cpu"

In [2]:
preprocess_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [3]:
train_ratio = 0.8

train_dataset = mnist.MNIST(root="./data", train=True, download=False, transform=preprocess_train)
test_dataset = mnist.MNIST(root="./data", train=False, download=False, transform=preprocess_train)

train_size = int(train_ratio * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, (train_size, val_size))

In [14]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(128*7*7, 64)
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        # 1x28x28
        # 1st layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)  #64x14x14

        # 2nd layer
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool(x) # 128x7x7

        # x = self.gap(x) # 128x1x1
        x = x.reshape(x.size(0), -1)

        # Fully Connected layer
        x = self.fc1(x)
        output = self.fc2(x)
        return output

In [15]:
learning_rate = 0.01

model = MNISTCNN().to("cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
batch_size=64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
number_of_epoch = 20
model.train()

for epoch in range(1, number_of_epoch+1):
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        # FP
        outputs = model(images)
        loss= criterion(outputs, labels)
        # BP
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += images.size(0)

    print(f"Epoch: {epoch}/{number_of_epoch}, Loss: {running_loss / len(train_dataloader)}, Accuracy: {correct / total}")

Epoch: 1/20, Loss: 0.4479100518375635, Accuracy: 0.8804166666666666
Epoch: 2/20, Loss: 0.24162058102587858, Accuracy: 0.9239166666666667
Epoch: 3/20, Loss: 0.2220906640489896, Accuracy: 0.9300416666666667
Epoch: 4/20, Loss: 0.2103060322354237, Accuracy: 0.9347291666666667
Epoch: 5/20, Loss: 0.20258984697858493, Accuracy: 0.9363541666666667
Epoch: 6/20, Loss: 0.2063052547921737, Accuracy: 0.9361458333333333
Epoch: 7/20, Loss: 0.1716219196021557, Accuracy: 0.9466041666666667
Epoch: 8/20, Loss: 0.1534860004025201, Accuracy: 0.9525416666666666
Epoch: 9/20, Loss: 0.1398270143835495, Accuracy: 0.9575
Epoch: 10/20, Loss: 0.1217917125041907, Accuracy: 0.9624791666666667
Epoch: 11/20, Loss: 0.11533743335182468, Accuracy: 0.9645833333333333
Epoch: 12/20, Loss: 0.10834665813110769, Accuracy: 0.9671041666666667
Epoch: 13/20, Loss: 0.10393973960075527, Accuracy: 0.968875
Epoch: 14/20, Loss: 0.09973819935360613, Accuracy: 0.9698958333333333
Epoch: 15/20, Loss: 0.09851686878463564, Accuracy: 0.969687

In [20]:
torch.save(model.state_dict(), "cnn_mnist.pth")

In [18]:
running_loss = 0
correct = 0
total = 0
for images, labels in val_dataloader:
    images, labels = images.to(device), labels.to(device)
    # FP
    outputs = model(images)
    loss= criterion(outputs, labels)
    # BP
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = outputs.max(1)
    correct += predicted.eq(labels).sum().item()
    total += images.size(0)

print(f"Epoch: {epoch}/{number_of_epoch}, Loss: {running_loss / len(val_dataloader)}, Accuracy: {correct / total}")

Epoch: 20/20, Loss: 0.11072797982512597, Accuracy: 0.9675


In [19]:
running_loss = 0
correct = 0
total = 0
for images, labels in test_dataloader:
    images, labels = images.to(device), labels.to(device)
    # FP
    outputs = model(images)
    loss= criterion(outputs, labels)
    # BP
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = outputs.max(1)
    correct += predicted.eq(labels).sum().item()
    total += images.size(0)

print(f"Epoch: {epoch}/{number_of_epoch}, Loss: {running_loss / len(test_dataloader)}, Accuracy: {correct / total}")

Epoch: 20/20, Loss: 0.11158064691703744, Accuracy: 0.9677


In [19]:
total

12000

In [16]:
total

12000